In [247]:
import numpy as np
import yaml
import pandas as pd    

from dataclasses import dataclass
from typing import List, Optional
import itertools
from docopt import docopt
from prettytable import PrettyTable 
from rich.prompt import Prompt
from datetime import datetime, timedelta

In [248]:
import logging
logging.basicConfig()
LOGGER = logging.getLogger(__name__)
LOGGER.setLevel(logging.INFO)


In [384]:
start_date

datetime.datetime(2022, 11, 27, 13, 57, 37, 963201)

In [249]:
def calculate_1rm(reps, weight, rpe):
    """
    Use Brzycki's formula to calculate the theortical 1RM
    """
    one_rm_pct = DF_RPE[f'{reps}'].to_dict().get(rpe)
    one_rm = int(np.round(np.floor(weight / one_rm_pct), 0))
    return one_rm

def round_weights(weight, deload=False):
    """
    Round the weights to the nearest 2.5 kg (or kg if microplates available)
    """
    if deload: 
        pl_weight = MIN_PLATE_WEIGHT * math.floor(weight / MIN_PLATE_WEIGHT)
    else: 
        pl_weight = MIN_PLATE_WEIGHT * math.ceil(weight / MIN_PLATE_WEIGHT)

    return pl_weight
            

In [385]:

@dataclass
class OneRepMax:
    """A dataclass to store the one rep max for a lift"""

    lift: str
    weight: float
        
    @property
    def output_table(self) -> str:
        """Return a table of the one rep max"""
        output = [self.lift.title(), self.weight]
        return output


@dataclass
class LiftWeight:
    """A dataclass to store the weight for a lift"""
    weight: float
    reps: int

    
@dataclass
class Exercise: 
    """A generic class for an exercise"""
    backoff_sets: List[LiftWeight]
    name: str
    predicted_one_rm: float = None
    related_compound: str = None
    top_sets: Optional[List[LiftWeight]] = None
    
    @property 
    def pretty_name(self) -> str:
        """Clean the names / related compound names"""
        exercise_name = self.name.replace('-', ' ').title()
        return exercise_name

    @property 
    def weekly_sets(self) -> List[LiftWeight]:
        """
        Return the weekly training set for the exercise
        """
        if self.top_sets:
            weekly_sets = list(itertools.zip_longest(self.top_sets, self.backoff_sets))
            return weekly_sets
        else: 
            return self.backoff_sets
    @property 
    def output_table(self) -> List: 
        "Creates table output for prettytable package"
        outputs = [f"**{self.pretty_name}**"]
        if self.top_sets:
            for _, (top, backoff) in enumerate(self.weekly_sets, 1):
                output = f"{top.weight} kg x {top.reps}\n"+\
                f"{backoff.weight} kg x {backoff.reps}"
                outputs.append(output)
        else: 
            for _, weight in enumerate(self.weekly_sets, 1):
                output = f"{weight.weight} kg x {weight.reps}"
                outputs.append(output)
        outputs.append('\n')                
        return outputs
    

In [252]:
def get_one_rm_pct(reps, rpe): 
    "Get the percentage of 1RM for a given number of reps and RPE"
    one_rm_pct = DF_RPE[f'{reps}'].to_dict().get(rpe)
    return one_rm_pct

In [253]:
def calculate_training_range(one_rm, reps, rpe_schema) -> List[float]:
    """
    Calculate the training day progression for a given 1RM
    """
    week_weights = []
    
    for rpe in rpe_schema:
        pct_one_rm = get_one_rm_pct(reps, rpe)
        week_weight = pct_one_rm * one_rm
        week_weights.append(week_weight)

    training_range = [round_weights(i) for i in week_weights]
        
    return training_range

In [383]:
proposal = [40.0, 42.5, 42.5, 45.0, 45.0]
weekly_difference = np.diff(proposal).tolist()
for i, weekly_diff in enumerate(weekly_difference, 0): 
    if weekly_diff < 2.5: 
        proposal[i+1] = proposal[i+1] + 2.5
        # update weekly difference value 
        weekly_difference = np.diff(proposal).tolist()
proposal

[40.0, 42.5, 45.0, 45.0, 47.5]

In [255]:
def update_weekly_difference(proposed_training): 
    "Ensures there is a minimum gap as specified by the user's gym"
    weekly_difference = np.diff(proposed_training).tolist()
    for i, weekly_diff in enumerate(weekly_difference):
        if weekly_diff < MIN_WEEKLY_DIFFERENCE:
            proposed_training[i+1] = proposed_training[i+1] + MIN_WEEKLY_DIFFERENCE
    return proposed_training

In [256]:
def calculate_training_range(one_rm, reps, rpe_schema) -> List[float]:
    """
    Calculate the training day progression for a given 1RM
    """
    lowest_rpe = rpe_schema[0]
    highest_rpe = rpe_schema[-1]

    w1_weight = round_weights(get_one_rm_pct(reps, lowest_rpe) * one_rm, deload=True)
    w5_weight = get_one_rm_pct(reps, highest_rpe) * one_rm 

    
    week_weights = np.linspace(w1_weight, w5_weight, 5).tolist()
    print(week_weights)
    training_range = [round_weights(i) for i in week_weights]
    print(training_range)

    # check if there is minimum spacing between each week 
    
    return training_range

In [257]:
def create_training_range(one_rm, reps, rpe_schema) -> List[LiftWeight]: 
    """Create a list of LiftWeight objects for the training range"""
    training_range = calculate_training_range(one_rm, reps, rpe_schema)
            
    weights_lifted = list(zip(training_range, itertools.repeat(reps)))
    weights_lifted = [LiftWeight(*i) for i in weights_lifted]

    return weights_lifted

In [258]:
def create_weekly_dates(start_date): 
    """
    Create the weekly dates for the program
    """
    starting_monday = datetime.date(
        start_date + timedelta(days=7 - start_date.weekday())
    )
    
    week_mondays = []
    for week in range(1, 6):
        week_monday = starting_monday + timedelta(days=7*week)
        week_mondays.append(week_monday)

    # Create the week dates as str 
    week_dates = [i.strftime('%d/%m/%y') for i in week_mondays]                
    return week_dates

In [386]:

if __name__ == "__main__":
    # arguments = docopt(__doc__, version="simple-strength-program v1.0")

    # # gather inputs
    # MICROPLATES = False

    # if arguments["--user"]:
    #     # ask for user profile
    #     user_profile = Prompt.ask("Enter user profile name", default="default")
    #     user_profile = user_profile.lower()
    #     user_profile = user_profile.replace(" ", "-")
    # else:
    #     user_profile = "default"
    user_profile='default'
    
    start_date=datetime.now()

    LOGGER.info("Reading source data and configs")
    defined_rpes = yaml.load(open("source/rpe.yaml", "r"), Loader=yaml.FullLoader)
    exercises = yaml.load(open("config/exercise.json", "r"), Loader=yaml.FullLoader)
    program_layout = yaml.load(open("config/program-layout.yaml", "r"), Loader=yaml.FullLoader)
    accessory_rpe_schema = defined_rpes.get("accessory").get("backoff")

    LOGGER.info("Reading user inputs")
    user_lifts = yaml.load(
        open(f"profiles/{user_profile}/lifts.yaml", "r"), Loader=yaml.FullLoader
    )
    try:
        user_gym = yaml.load(
            open(f"profiles/{user_profile}/gym.yaml", "r"), Loader=yaml.FullLoader
        )
    except FileNotFoundError:
        user_gym = False


    # Global variables
    DF_RPE = pd.read_csv("source/rpe-calculator.csv").set_index("RPE")
    MICROPLATES = user_gym.get("microplates") if user_gym else False
    MIN_PLATE_WEIGHT = 1 if MICROPLATES else 2.5
    WEEK_DATES = create_weekly_dates(start_date)
    
    # create weekly dates 
    LOGGER.info("Creating weekly dates")


    max_lifts = []
    full_program = []

    for compound_lift_name, stats in user_lifts.items():
        # clear previous stats
        LOGGER.info("Creating program for %s", compound_lift_name)
        top_training_range = []
        backoff_training_range = []

        lift_one_rep_max = calculate_1rm(
            reps=stats.get("reps"), rpe=stats.get("rpe"), weight=stats.get("weight")
        )

        max_strength = OneRepMax(lift=compound_lift_name, weight=lift_one_rep_max)

        goal = stats.get("program-goal")
        program_type = goal.split("-")[0]
        rpes = defined_rpes.get(program_type)
        exercise_volume = exercises.get("program-goals").get(f"{goal}")

        if program_type == "strength":
            top_training_range = create_training_range(
                    one_rm=lift_one_rep_max,
                    reps=exercise_volume.get("top-reps"),
                    rpe_schema=rpes.get("top"),
                )
            

        backoff_training_range = create_training_range(
            one_rm=lift_one_rep_max,
            reps=exercise_volume.get("backoff-reps"),
            rpe_schema=rpes.get("backoff"),
        )

        compound_lift = Exercise(
            backoff_sets=backoff_training_range,
            name=compound_lift_name,
            predicted_one_rm=lift_one_rep_max,
            top_sets=top_training_range,
        )

        full_program.append(compound_lift)

        accessories = exercises.get("accessory-lifts").get(f"{compound_lift_name}")
        for accessory, stats in accessories.items():
            accessory_one_rm = round_weights(
                lift_one_rep_max * stats.get("max-onerm-pct")
            )
            min_reps = stats.get("min-reps")
            max_reps = stats.get("max-reps")

            accessory_training_range = create_training_range(
                one_rm=accessory_one_rm, reps=min_reps, rpe_schema=accessory_rpe_schema
            )

            accessory_lift = Exercise(
                backoff_sets=accessory_training_range,
                name=accessory,
                predicted_one_rm=accessory_one_rm,
                related_compound=compound_lift,
            )

            full_program.append(accessory_lift)

        max_lifts.append(max_strength)
    
    # create the program
    LOGGER.info("Creating program")
    all_exercises = {}
    for exercise in full_program: 
        print(exercise)
        all_exercises[exercise.name] = exercise

    

INFO:__main__:Reading source data and configs
INFO:__main__:Reading user inputs
INFO:__main__:Creating weekly dates
INFO:__main__:Creating program for squat
INFO:__main__:Creating program for bench
INFO:__main__:Creating program for deadlift
INFO:__main__:Creating program


[125.0, 128.709125, 132.41825, 136.127375, 139.8365]
[125.0, 130.0, 132.5, 137.5, 140.0]
[112.5, 115.55324999999999, 118.6065, 121.65975, 124.713]
[112.5, 117.5, 120.0, 122.5, 125.0]
[97.5, 100.49625, 103.4925, 106.48875000000001, 109.48500000000001]
[97.5, 102.5, 105.0, 107.5, 110.0]
[102.5, 105.3075, 108.11500000000001, 110.9225, 113.73]
[102.5, 107.5, 110.0, 112.5, 115.0]
[50.0, 51.7875, 53.575, 55.3625, 57.15]
[50.0, 52.5, 55.0, 57.5, 57.5]
[95.0, 98.01, 101.02000000000001, 104.03, 107.04]
[95.0, 100.0, 102.5, 105.0, 107.5]
[75.0, 77.7765, 80.553, 83.3295, 86.106]
[75.0, 80.0, 82.5, 85.0, 87.5]
[75.0, 76.8775, 78.755, 80.63250000000001, 82.51]
[75.0, 77.5, 80.0, 82.5, 85.0]
[62.5, 64.715, 66.93, 69.145, 71.36]
[62.5, 65.0, 67.5, 70.0, 72.5]
[72.5, 74.331875, 76.16375, 77.995625, 79.8275]
[72.5, 75.0, 77.5, 80.0, 80.0]
[12.5, 13.45625, 14.4125, 15.368749999999999, 16.325]
[12.5, 15.0, 15.0, 17.5, 17.5]
[22.5, 23.405, 24.310000000000002, 25.215, 26.12]
[22.5, 25.0, 25.0, 27.5, 27.5]


In [374]:
WEEK_DATES.insert(0, "Week Starting")
WEEK_DATES 

['Week Starting', '05/12/22', '12/12/22', '19/12/22', '26/12/22', '02/01/23']

In [399]:
program_layout.values()

dict_values([['squat', 'paused-bench', 'romanian-deadlift', 'bulgarian-split-squat', 'superman-ab-crunch'], ['bench', 'paused-deadlift', 'face-pulls', 'tricep-pushdowns'], ['deadlift', 'safety-squat-bar', 'bulgarian-split-squat', 'incline-bench-press', 'bicep-curls']])

In [332]:
user_stats = PrettyTable()
user_stats.field_names = ['Lift', 'Training Max']
for lift in max_lifts:
    max_tm_row = lift.output_table
    user_stats.add_row(max_tm_row)

user_stats

Lift,Training Max
Squat,149
Bench,113
Deadlift,209


In [406]:
max_exercises = len(max(list(program_layout.values()), key=len))

In [358]:
def column_pad(*columns):
  max_len = max([len(c) for c in columns])
  for c in columns:
      c.extend(['']*(max_len-len(c)))

['Actual Load',
 '',
 '',
 '',
 '',
 '',
 '',
 'Actual Load',
 '',
 '',
 '',
 '',
 '',
 '',
 'Actual Load',
 '',
 '',
 '',
 '',
 '',
 '',
 'Actual Load',
 '',
 '',
 '',
 '',
 '',
 '',
 'Actual Load',
 '',
 '',
 '',
 '',
 '',
 '']

In [392]:
day_cols = []
for _, exercises in program_layout.items(): 
    day_col = []
    for exercise_name in exercises: 
        planned_exercise = all_exercises.get(exercise_name)
        if planned_exercise: 
            exercise_col = planned_exercise.output_table
        else:
            generic_exericse=Exercise(
                backoff_sets= [LiftWeight(0, 12)] * 5,
                name=exercise_name,
            )
            exercise_col = generic_exericse.output_table
        day_col.append(exercise_col)
    flatten_day_col = [item for sublist in day_col for item in sublist]

    day_cols.append(flatten_day_col)

column_pad(*day_cols)

program = PrettyTable()
week_dates=create_weekly_dates(start_date)
week_dates.insert(0, "Week Starting")
week_dates.append("")
five_week_dates= week_dates * 5
program.add_column("", five_week_dates)

for day, col in enumerate(day_cols, 1): 
    program.add_column(f"Day {day}", col)


In [393]:
# create weekly dates

program

,Day 1,Day 2,Day 3
Week Starting,**Squat**,**Bench**,**Deadlift**
05/12/22,125.0 kg x 2112.5 kg x 4,75.0 kg x 8,140.0 kg x 8
12/12/22,130.0 kg x 2117.5 kg x 4,80.0 kg x 8,145.0 kg x 8
19/12/22,132.5 kg x 2120.0 kg x 4,82.5 kg x 8,150.0 kg x 8
26/12/22,137.5 kg x 2122.5 kg x 4,85.0 kg x 8,155.0 kg x 8
02/01/23,140.0 kg x 2125.0 kg x 4,87.5 kg x 8,160.0 kg x 8
,,,
Week Starting,**Paused Bench**,**Paused Deadlift**,**Safety Squat Bar**
05/12/22,75.0 kg x 3,127.5 kg x 3,102.5 kg x 3
12/12/22,77.5 kg x 3,132.5 kg x 3,107.5 kg x 3


In [381]:
user_stats

Lift,Training Max
Squat,149
Bench,113
Deadlift,209
